# Number classification using Kaggle Cdiscount dataset

In [ ]:
# Function to identify environment
def checkIsKaggle ():
    import os 
    return 'KAGGLE_URL_BASE' in os.environ 
isKaggle = checkIsKaggle()
print("This is a Kaggle environment" if isKaggle else "This is not a Kaggle environment")



In [ ]:
# Check GPU compatibility with Pytorch and print out GPU information
import torch
if torch.cuda.is_available():
    print("GPU is available")
    print("GPU device:", torch.cuda.get_device_name(0))
    print("GPU memory:", round(torch.cuda.get_device_properties(0).total_memory/1024**3),"GB")
else:
    print("GPU is not available. Using CPU")
    
CUDA = torch.cuda.is_available()

## 1. Import libraries and models

In [ ]:

import io 
import torch # Pytorch
from torch import nn # Pytorch's neural network module
import bson # handle BSON file format
import numpy as np # Numpy
from skimage.io import imread   
import matplotlib.pyplot as plt # Matplotlib plot
import h5py # Handle HDF5 file format
from glob import glob
from tqdm import tqdm # Progress bar

if isKaggle:
    pass
else:
    from models.__VGG16__ import VGG16 # VGG16 model


In [ ]:
# if isKaggle:
#     pass
# else:
#     torch.save(VGG16, "./models/VGG16.pt")
    
#     summary(VGG16(1000).cuda(), (3, 224, 224)) # Print the model summary

## 2. Process data from bson


In [ ]:
bson_dir = "/kaggle/input" if isKaggle else "./data" # Input directory

processed_dir = "/tmp/processed_data" if isKaggle else "./processed_data" 

normalized_dir = "/tmp/normalized_data" if isKaggle else "./normalized_data"

def processBson(filename:str) -> None:
    filenameWithoutExt = filename.split(".")[0] # Get filename without extension
    
    data = bson.decode_file_iter(open(f"{bson_dir}/{filename}","rb")) # Open BSON file
    
    f = h5py.File(f"{processed_dir}/{filenameWithoutExt}.hdf5", "w") # Open HDF5 file
    
    f.create_dataset(name="image",shape=(0,280,280,3),dtype="i",chunks=True,maxshape=(None,280,280,3) ) # Create dataset for images 
    f.create_dataset(name="category_id",shape=(0,),dtype="i",chunks=True,maxshape=(None,)) # Create dataset for labels
    f.create_dataset(name="product_id",shape=(0,),dtype="i",chunks=True,maxshape=(None,)) # Create dataset for product ID
    
    image_array = [] # Image list
    category_id_array = [] # Label list
    product_id_array = [] # Product ID list
    
    def flushData():
        pass
    
    def appendData(image,product_id,category_id):
        image_array.append(image)
        product_id_array.append(product_id)
        category_id_array.append(category_id)
        if len(image_array) == 1000:
            flushData()
    
    for product in tqdm(data):
        product_id = product['_id'] # Get product ID
        category_id = product['category_id'] if filenameWithoutExt != "test" else -1 # Get category ID
        for pic in product['imgs']:
            img = imread(io.BytesIO(pic['picture'])) # Read image
    
    flushData()
    f.close() # Close HDF5 file
            
            
processBson("test.bson")
for filename in glob(f"{bson_dir}/*.bson"):
    print(filename.split("\\")[-1])
    
    
    

